In [1]:
# 匯入套件
import requests
import pandas as pd
import json
import re
from lxml import etree
import time
#執行函式
BRAND = "淨妍醫美診所"
WEBSITE_URL = "https://www.jinyenclinic.com"


def get_doctor_detail(url):
    resp = requests.get(url)
    root = etree.HTML(resp.content.decode('utf-8'))
    result = {
        "brand":BRAND,  
        "pic":"".join(root.xpath("//section[@class='single-page-section dr']//img/@src")), 
        "doctor_name":"".join(root.xpath("//section[@class='single-page-section dr']//img/@alt")),
#         "store_name":root.xpath("(//div[contains(@class,'cont')])[1]/text()"),
#         "city":"",
#         "address":"",
#         "tel":"",
#         "personal_website":"",
#         "store_website":"",
#         "specialty":root.xpath("(//div[contains(@class,'cont')])[5]/text()"),
#         "description":"",
#         "case_study":"",
         "experience":root.xpath("(//section[@class='single-page-section dr']//div)[6]//text()"),
         "education":root.xpath("(//section[@class='single-page-section dr']//div)[5]//text()") 
    }
    return result
def get_doctors():
    result = []
    resp = requests.get( f"{WEBSITE_URL}/DoctorAll")
    root = etree.HTML(resp.content.decode('utf-8'))
    doctor_urls = root.xpath("//section[@class='single-page-section dr-all']//a/@href")
    return list(map(lambda i: f"{WEBSITE_URL}/{i}", doctor_urls))

doctor_list = get_doctors()
doctor_details = []
for idx,url in enumerate(doctor_list):
    print(f"{idx+1}/{len(doctor_list)}, {url}")
    doctor_details.append(get_doctor_detail(url))
    
for fir in range(len(doctor_details)):
    for sec in range(len(doctor_details[fir]['education'])):
        if doctor_details[fir]['education'][sec]=='/ 學歷 /':
            doctor_details[fir]['education'][sec]=doctor_details[fir]['education'][sec].replace('/ 學歷 /','')
        elif doctor_details[fir]['education'][sec]=='\n':
            doctor_details[fir]['education'][sec]=doctor_details[fir]['education'][sec].replace('\n','')
    while '' in doctor_details[fir]['education']:
        doctor_details[fir]['education'].remove('')
    doctor_details[fir]['education']=",".join(doctor_details[fir]['education'])
    for sec in range(len(doctor_details[fir]['experience'])):
        if doctor_details[fir]['experience'][sec]=='/ 經歷 /':
            doctor_details[fir]['experience'][sec]=doctor_details[fir]['experience'][sec].replace('/ 經歷 /','')
        elif doctor_details[fir]['experience'][sec]=='\n':
            doctor_details[fir]['experience'][sec]=doctor_details[fir]['experience'][sec].replace('\n','')
        doctor_details[fir]['experience']=[sub.replace('\xa0', '').replace('●', '').replace('\n', '') for sub in doctor_details[fir]['experience']]
    while '' in doctor_details[fir]['experience']:
        doctor_details[fir]['experience'].remove('')
    doctor_details[fir]['experience']=",".join(doctor_details[fir]['experience'])
    
#執行程式
FILE_NAME = BRAND
today = time.strftime("%Y%m%d")
df = pd.DataFrame(doctor_details, columns=["brand","pic","doctor_name","store_name","city","address","tel","personal_website","store_website","specialty","description","case_study","experience","education"])
df.to_csv(f"{FILE_NAME}-{today}.csv",index=False)